In [ ]:
# Run in Google Colab (Python 3.10) or local Python (>=3.8).
# Writes CSV to ./ec_lwe_benchmark_corrected.csv

import time
import secrets
import hashlib
import math
import csv
from statistics import mean, stdev

# -----------------------
# Curve parameters (secp256k1-like)
# -----------------------
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
a = 0
b = 7
# generator (decimal from secp256k1)
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
P = (Gx, Gy)
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141  # order

# -----------------------
# Field and EC arithmetic (basic; not constant-time)
# -----------------------
def mod_inv(x):
    return pow(x % p, p - 2, p)

def ec_add(P1, P2):
    if P1 is None:
        return P2
    if P2 is None:
        return P1
    x1, y1 = P1
    x2, y2 = P2
    if x1 == x2 and (y1 + y2) % p == 0:
        return None
    if P1 == P2:
        lam = (3 * x1 * x1 + a) * mod_inv(2 * y1) % p
    else:
        lam = (y2 - y1) * mod_inv((x2 - x1) % p) % p
    x3 = (lam * lam - x1 - x2) % p
    y3 = (lam * (x1 - x3) - y1) % p
    return (x3, y3)

def ec_mul(k, P):
    if k % n == 0 or P is None:
        return None
    if k < 0:
        # negative multiplication: k * P = -k * (-P)
        return ec_mul(-k, (P[0], (-P[1]) % p))
    R = None
    Q = P
    while k:
        if k & 1:
            R = ec_add(R, Q)
        Q = ec_add(Q, Q)
        k >>= 1
    return R

# -----------------------
# Point compression / decompression (secp256k1-style)
# compressed: 0x02 or 0x03 + x (32 bytes)
# uncompressed: 0x04 + x + y
# -----------------------
def compress_point(Pt):
    x, y = Pt
    prefix = b'\x02' if (y % 2 == 0) else b'\x03'
    return prefix + x.to_bytes(32, 'big')

def decompress_point(data):
    # expects compressed 33-byte input
    if len(data) == 65 and data[0] == 0x04:
        x = int.from_bytes(data[1:33], 'big')
        y = int.from_bytes(data[33:65], 'big')
        return (x, y)
    if len(data) != 33:
        raise ValueError("Invalid compressed point length")
    prefix = data[0]
    x = int.from_bytes(data[1:33], 'big')
    # solve y^2 = x^3 + ax + b (mod p)
    rhs = (pow(x, 3, p) + a * x + b) % p
    # Tonelli-Shanks variant for p % 4 == 3 (true for secp256k1)
    y = pow(rhs, (p + 1) // 4, p)
    if (y % 2 == 0 and prefix == 0x02) or (y % 2 == 1 and prefix == 0x03):
        return (x, y)
    else:
        return (x, (-y) % p)

# -----------------------
# RFC-like expand_message_xmd (XMD-SHA256) per RFC 9380 (prototype)
# -----------------------
def i2osp(x, length):
    return x.to_bytes(length, 'big')

def os2ip(b):
    return int.from_bytes(b, 'big')

def xor_bytes(a, b):
    return bytes(x ^ y for x, y in zip(a, b))

def expand_message_xmd(msg, dst, len_in_bytes):
    b_in_bytes = hashlib.sha256().digest_size
    r_in_bytes = b_in_bytes
    ell = (len_in_bytes + b_in_bytes - 1) // b_in_bytes
    if ell > 255:
        raise ValueError("expand_message_xmd: ell > 255")
    dst_prime = dst + i2osp(len(dst), 1)
    z_pad = b'\x00' * b_in_bytes
    l_i_b_str = i2osp(len_in_bytes, 2)
    b0 = hashlib.sha256(z_pad + msg + l_i_b_str + b'\x00' + dst_prime).digest()
    b_vals = []
    b1 = hashlib.sha256(b0 + b'\x01' + dst_prime).digest()
    b_vals.append(b1)
    for i in range(1, ell):
        bi = hashlib.sha256(xor_bytes(b0, b_vals[i-1]) + bytes([i+1]) + dst_prime).digest()
        b_vals.append(bi)
    pseudo_random = b''.join(b_vals)[:len_in_bytes]
    return pseudo_random

# -----------------------
# hash_to_field and Simplified SWU mapping (prototype, adapted)
# -----------------------
def hash_to_field(msg, count=2, dst=b'EC-LWE_XMD:SHA-256_SSWU_'):
    # returns list of field elements
    len_in_bytes = count * 32
    uniform_bytes = expand_message_xmd(msg, dst, len_in_bytes)
    els = []
    for i in range(count):
        tv = int.from_bytes(uniform_bytes[32*i:32*(i+1)], 'big') % p
        els.append(tv)
    return els

# Simplified SWU prototype mapping constants for secp256k1 (Z chosen)
Z = -11 % p

def sqrt_mod(a):
    # p % 4 == 3 for secp256k1, so sqrt is a^{(p+1)/4}
    return pow(a, (p + 1) // 4, p)

def map_to_curve_simple_swu(u):
    # Prototype: simplified SWU-ish mapping adapted for a=0 curve.
    # Not a drop-in replacement in production; for research/prototype use.
    # Aim: produce a curve point from u.
    tv1 = pow(u, 2, p)
    tv1 = (tv1 * Z) % p
    # x1 = (-b / (1 + tv1)) mod p
    denom = (1 + tv1) % p
    denom_inv = mod_inv(denom)
    x1 = (-b * denom_inv) % p
    gx1 = (pow(x1, 3, p) + a * x1 + b) % p
    if pow(gx1, (p - 1) // 2, p) == 1:
        y1 = sqrt_mod(gx1)
        return (x1, y1)
    # otherwise try x2
    x2 = (tv1 * x1) % p
    gx2 = (pow(x2, 3, p) + a * x2 + b) % p
    y2 = sqrt_mod(gx2)
    return (x2, y2)

def hash_to_curve_rfc9380(msg):
    u_vals = hash_to_field(msg, count=2)
    Q1 = map_to_curve_simple_swu(u_vals[0])
    Q2 = map_to_curve_simple_swu(u_vals[1])
    R = ec_add(Q1, Q2)
    # ensure not point at infinity (very unlikely)
    if R is None:
        # fallback: use generator
        return P
    return R

# -----------------------
# EC-LWE core operations (fixed)
# -----------------------
def key_generation():
    k_B = secrets.randbelow(n - 1) + 1
    Q_B = ec_mul(k_B, P)
    return k_B, Q_B

def encode_shared_point(S):
    # fixed-length encoding: 32-byte BE for x and y concatenated
    Sx, Sy = S
    return Sx.to_bytes(32, 'big') + Sy.to_bytes(32, 'big')

def encrypt_chunk_fixed(chunk_bytes, SID, nonce, Q_B, P):
    # chunk_bytes: bytes exactly 32 bytes (padded if needed)
    M_int = int.from_bytes(chunk_bytes, 'big') % p
    r = secrets.randbelow(n - 1) + 1
    C1 = ec_mul(r, P)
    S = ec_mul(r, Q_B)
    shared_bytes = encode_shared_point(S) + SID.encode() + nonce
    K = hashlib.sha256(shared_bytes).digest()
    e = hash_to_curve_rfc9380(K)
    C2 = (M_int + e[0]) % p
    # return compressed C1 and C2 as integer
    C1_ser = compress_point(C1)  # 33 bytes
    return C1_ser, C2, r

def decrypt_chunk_fixed(C1_point, C2, k_B, SID, nonce):
    # C1_point is a tuple (x,y)
    S = ec_mul(k_B, C1_point)
    shared_bytes = encode_shared_point(S) + SID.encode() + nonce
    K = hashlib.sha256(shared_bytes).digest()
    e = hash_to_curve_rfc9380(K)
    M_int = (C2 - e[0]) % p
    # return fixed-length 32-byte chunk
    return M_int.to_bytes(32, 'big')

# -----------------------
# Helper: chunking with deterministic padding
# -----------------------
def chunk_message(msg_bytes, chunk_size=32):
    chunks = []
    for i in range(0, len(msg_bytes), chunk_size):
        chunk = msg_bytes[i:i+chunk_size]
        if len(chunk) < chunk_size:
            # deterministic padding: pad with zeros (note length must be tracked externally)
            chunk = chunk + b'\x00' * (chunk_size - len(chunk))
        chunks.append(chunk)
    return chunks

# -----------------------
# Benchmark driver
# -----------------------
def warmup_iterations(Q_B, P, iterations=8):
    # warm up scalar mul and hash-to-curve
    for _ in range(iterations):
        k = secrets.randbelow(n - 1) + 1
        _ = ec_mul(k, P)
        # small dummy hash->curve
        dummy = hashlib.sha256(b'warmup' + secrets.token_bytes(8)).digest()
        _ = hash_to_curve_rfc9380(dummy)

def run_benchmark(message_sizes=[16,32,64,128,256,512,1024], runs_per_size=50, output_csv='./ec_lwe_benchmark_corrected.csv'):
    headers = ["Scheme", "Message Size", "Run", "KeyGen Time", "Enc Time", "Dec Time", "Ciphertext Size"]
    records = []

    for size in message_sizes:
        # create a single message to encrypt per run but with fresh SID & nonces per run
        for run in range(1, runs_per_size + 1):
            msg = ''.join(secrets.choice("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789") for _ in range(size)).encode()
            # ensure msg length is size bytes
            assert len(msg) == size
            SID = secrets.token_hex(8)  # session id string
            # key generation timing (measured properly)
            t0 = time.perf_counter()
            k_B, Q_B = key_generation()
            keygen_time = time.perf_counter() - t0

            # warmup (only once per run to avoid contaminating timing)
            warmup_iterations(Q_B, P, iterations=4)

            # chunk into 32-byte chunks with deterministic padding
            chunks = chunk_message(msg, 32)
            ciphertext_chunks = []

            # encryption
            t0 = time.perf_counter()
            # generate starting nonce as integer, then increment for each chunk
            nonce0_int = int.from_bytes(secrets.token_bytes(16), 'big')
            for i, chunk in enumerate(chunks, start=1):
                nonce_i_int = (nonce0_int + (i - 1)) % (1 << (16*8))  # 16-byte space
                nonce_i = nonce_i_int.to_bytes(16, 'big')
                C1_ser, C2, r = encrypt_chunk_fixed(chunk, SID, nonce_i, Q_B, P)
                ciphertext_chunks.append((C1_ser, C2, nonce_i))
            enc_time = time.perf_counter() - t0

            # decryption
            t0 = time.perf_counter()
            decrypted_chunks = []
            for (C1_ser, C2, nonce_i) in ciphertext_chunks:
                C1_point = decompress_point(C1_ser)
                plaintext_chunk = decrypt_chunk_fixed(C1_point, C2, k_B, SID, nonce_i)
                decrypted_chunks.append(plaintext_chunk)
            # recovered bytes: strip the deterministic padding to original message length
            recovered = b''.join(decrypted_chunks)[:len(msg)]
            dec_time = time.perf_counter() - t0

            # ciphertext size: compressed C1 (33) + C2 (32) per chunk
            per_chunk_ct_size = 33 + 32
            ciphertext_size = len(ciphertext_chunks) * per_chunk_ct_size

            status = "SUCCESS" if recovered == msg else "FAIL"
            if status != "SUCCESS":
                print("Decryption failed for size", size, "run", run)

            records.append(["EC-LWE (RFC9380)", size, run, keygen_time, enc_time, dec_time, ciphertext_size])

            # optional short progress print
            if run % 10 == 0:
                print(f"size {size} run {run}: keygen {keygen_time:.6f}s enc {enc_time:.6f}s dec {dec_time:.6f}s ct {ciphertext_size}B")
    # write CSV and summary stats
    with open(output_csv, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(records)
    print(f"Benchmark complete. CSV written to {output_csv}")
    return records

# -----------------------
# Aggregation helper to compute per-size means/stdev/min/max and write summary CSV
# -----------------------
def aggregate_and_write(records, out_summary_csv='./ec_lwe_benchmark_summary.csv'):
    # records: list of rows as appended in run_benchmark
    from collections import defaultdict
    groups = defaultdict(list)
    for rec in records:
        scheme, size, run, keygen, enc, dec, ct = rec
        groups[size].append((keygen, enc, dec, ct))
    summary_rows = [["Message Size", "KeyGen Mean", "KeyGen Std", "KeyGen Min", "KeyGen Max",
                     "Enc Mean", "Enc Std", "Enc Min", "Enc Max",
                     "Dec Mean", "Dec Std", "Dec Min", "Dec Max", "Ciphertext (B)"]]
    for size in sorted(groups.keys()):
        vals = groups[size]
        keygens = [v[0] for v in vals]
        encs = [v[1] for v in vals]
        decs = [v[2] for v in vals]
        cts = vals[0][3]  # ciphertext size same for all runs of that size
        summary_rows.append([
            size,
            f"{mean(keygens):.6f}", f"{(stdev(keygens) if len(keygens)>1 else 0):.6f}", f"{min(keygens):.6f}", f"{max(keygens):.6f}",
            f"{mean(encs):.6f}", f"{(stdev(encs) if len(encs)>1 else 0):.6f}", f"{min(encs):.6f}", f"{max(encs):.6f}",
            f"{mean(decs):.6f}", f"{(stdev(decs) if len(decs)>1 else 0):.6f}", f"{min(decs):.6f}", f"{max(decs):.6f}",
            cts
        ])
    with open(out_summary_csv, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(summary_rows)
    print(f"Summary CSV written to {out_summary_csv}")

# -----------------------
# Main
# -----------------------
if __name__ == "__main__":
    # For quicker debugging set runs_per_size low (e.g., 5). For final results set to 50.
    MESSAGE_SIZES = [16, 32, 64, 128, 256, 512, 1024]
    RUNS_PER_SIZE = 50  # change for quicker test
    RAW_CSV = './ec_lwe_result.csv'
    SUMMARY_CSV = './ec_lwe_result_summary.csv'

    recs = run_benchmark(message_sizes=MESSAGE_SIZES, runs_per_size=RUNS_PER_SIZE, output_csv=RAW_CSV)
    aggregate_and_write(recs, out_summary_csv=SUMMARY_CSV)
    print("Done.")


size 16 run 10: keygen 0.047384s enc 0.090514s dec 0.045936s ct 65B
size 16 run 20: keygen 0.046596s enc 0.098494s dec 0.046741s ct 65B
size 16 run 30: keygen 0.048076s enc 0.095095s dec 0.052226s ct 65B
size 16 run 40: keygen 0.048608s enc 0.096768s dec 0.049798s ct 65B
size 16 run 50: keygen 0.049086s enc 0.113908s dec 0.051654s ct 65B
size 32 run 10: keygen 0.049003s enc 0.150637s dec 0.053570s ct 65B
size 32 run 20: keygen 0.044292s enc 0.086734s dec 0.046015s ct 65B
size 32 run 30: keygen 0.063733s enc 0.103074s dec 0.047978s ct 65B
size 32 run 40: keygen 0.053334s enc 0.103307s dec 0.051330s ct 65B
size 32 run 50: keygen 0.043968s enc 0.144948s dec 0.049675s ct 65B
size 64 run 10: keygen 0.051809s enc 0.303547s dec 0.144538s ct 130B
size 64 run 20: keygen 0.047525s enc 0.235666s dec 0.127968s ct 130B
size 64 run 30: keygen 0.050370s enc 0.232705s dec 0.097097s ct 130B
size 64 run 40: keygen 0.043619s enc 0.216939s dec 0.089263s ct 130B
size 64 run 50: keygen 0.049505s enc 0.21675

In [2]:
import time, secrets
from coincurve import PublicKey, PrivateKey

# Key generation
t0 = time.perf_counter()
priv = PrivateKey()
pub = priv.public_key
t1 = time.perf_counter()
print("ECC keygen:", t1 - t0)

# ECDH
peer = PrivateKey().public_key
t0 = time.perf_counter()
shared = priv.ecdh(peer.format())  # 32-byte shared secret
t1 = time.perf_counter()
print("ECC ECDH:", t1 - t0)


ECC keygen: 0.0002907998859882355
ECC ECDH: 0.0006547998636960983


In [3]:
import secrets, hashlib, math
from collections import Counter
import time
import numpy as np

# ============================================================
#  Elliptic-curve parameters (secp256k1)
# ============================================================
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
a, b = 0, 7

Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
P = (Gx, Gy)

# ============================================================
#  Field & EC ops
# ============================================================
def mod_inv(x): return pow(x, p-2, p)

def ec_add(P1, P2):
    if P1 is None: return P2
    if P2 is None: return P1
    x1,y1 = P1; x2,y2 = P2
    if x1 == x2 and (y1 + y2) % p == 0: return None
    if P1 == P2:
        lam = (3*x1*x1) * mod_inv(2*y1) % p
    else:
        lam = (y2 - y1) * mod_inv((x2 - x1) % p) % p
    x3 = (lam*lam - x1 - x2) % p
    y3 = (lam*(x1 - x3) - y1) % p
    return (x3,y3)

def ec_mul(k, P):
    R = None
    Q = P
    while k:
        if k & 1: R = ec_add(R, Q)
        Q = ec_add(Q, Q)
        k >>= 1
    return R

# ============================================================
# RFC 9380 Simplified SWU
# ============================================================
def hash_to_field(msg, count=2):
    dst = b"RFC9380_secp256k1_XMD:SHA-256_SSWU_RO_"
    # expand_message_xmd (correct construction)
    b_len = 64 * count
    b0 = hashlib.sha256(msg + dst).digest()
    bi = hashlib.sha256(b0 + dst).digest()
    uniform_bytes = (b0 + bi)[:b_len]

    els = []
    for i in range(count):
        tv = int.from_bytes(uniform_bytes[32*i:32*(i+1)], 'big') % p
        els.append(tv)
    return els

def map_to_curve_simple_swu(u):
    Z = -11 % p
    # SWU for a=0 curve
    tv1 = (u*u) % p
    tv1 = (tv1 * Z) % p
    x1 = (tv1 + 1) % p
    x1 = mod_inv(x1)
    x1 = (-b * x1) % p
    gx = (x1*x1*x1 + 7) % p
    y = pow(gx, (p+1)//4, p)
    return (x1, y)

def hash_to_curve_rfc9380(msg):
    u_vals = hash_to_field(msg)
    Q1 = map_to_curve_simple_swu(u_vals[0])
    Q2 = map_to_curve_simple_swu(u_vals[1])
    return ec_add(Q1, Q2)

# ============================================================
# Compression (33-byte SEC1)
# ============================================================
def compress_point(P):
    x, y = P
    return (b'\x02' if y % 2 == 0 else b'\x03') + x.to_bytes(32, 'big')

# ============================================================
# EC-LWE ops
# ============================================================
def key_generation():
    k = secrets.randbelow(p-1) + 1
    Q = ec_mul(k, P)
    return k, Q

def encrypt_single_chunk(chunk, SID, nonce, Q_B):
    M = int.from_bytes(chunk, 'big') % p
    r = secrets.randbelow(p-1) + 1

    C1 = ec_mul(r, P)
    S  = ec_mul(r, Q_B)

    shared = S[0].to_bytes(32,'big') + S[1].to_bytes(32,'big') + SID.encode() + nonce
    K = hashlib.sha256(shared).digest()
    e = hash_to_curve_rfc9380(K)

    C2 = (M + e[0]) % p
    return compress_point(C1), C2

# ============================================================
# Shannon Entropy
# ============================================================
def shannon_entropy(data_bytes):
    if len(data_bytes) == 0: return 0.0
    counts = Counter(data_bytes)
    total = len(data_bytes)
    probs = [c/total for c in counts.values()]
    return -sum(p * math.log2(p) for p in probs)

# ============================================================
# MAIN TEST
# ============================================================
def test_entropy(runs=100):
    SID = "Session123"
    nonce0 = secrets.token_bytes(16)
    nonce0_int = int.from_bytes(nonce0, 'big')

    message = b"HelloEC-LWE!!"  # <= small chunk, constant
    chunk = message

    k_B, Q_B = key_generation()

    all_C2 = []

    for i in range(runs):
        nonce_i = (nonce0_int + i).to_bytes(16, 'big')
        C1c, C2 = encrypt_single_chunk(chunk, SID, nonce_i, Q_B)

        # store C2 for uniqueness + entropy
        all_C2.append(C2.to_bytes(32, 'big'))

    flat_bytes = b''.join(all_C2)
    entropy = shannon_entropy(flat_bytes)

    unique_count = len(set(all_C2))

    print(f"Runs = {runs}")
    print(f"Unique C2 values = {unique_count}/{runs}")
    print(f"Entropy (bits/byte) = {entropy:.4f}")

# ============================================================
# RUN TESTS
# ============================================================
for r in [100, 200, 500, 1000]:
    test_entropy(r)
    print("------------------------------------------------")


Runs = 100
Unique C2 values = 100/100
Entropy (bits/byte) = 7.9469
------------------------------------------------
Runs = 200
Unique C2 values = 200/200
Entropy (bits/byte) = 7.9697
------------------------------------------------
Runs = 500
Unique C2 values = 500/500
Entropy (bits/byte) = 7.9897
------------------------------------------------
Runs = 1000
Unique C2 values = 1000/1000
Entropy (bits/byte) = 7.9934
------------------------------------------------
